In [1]:
def vec_from_by(ax,fro=[0,0],delta=[1,1],scale=1,col='black',label=''):
    ax.text(fro[0]+delta[0]/2,fro[1]+delta[1]/3,label,color=col,fontsize=20)
    return ax.arrow(fro[0], fro[1], delta[0], delta[1], head_width=0.1*scale, head_length=0.15*scale, color=col,length_includes_head=True)

In [1]:
def line_from_by(ax,fro=[0,0],delta=[1,1],scale=1,col='black',label='',ls='-'):
    ax.text(fro[0]+delta[0]/2*scale,fro[1]+delta[1]/3*scale,label,color=col,fontsize=20)
    return ax.arrow(fro[0], fro[1], delta[0]*scale, delta[1]*scale, head_width=0, head_length=0, color=col,length_includes_head=True,ls=ls)

In [2]:
def define_element(geo,ids):
    block = geo.loc[geo['Pkt'] < 0] #emtpy
    for i in ids:
        block = block.append(geo.loc[geo['Pkt'] == i])
    return block

In [5]:
def plot_blocks(blocks):
    fig, ax = plt.subplots()
    for i,block in enumerate(blocks):
        for k in range(len(block)):
            ax.plot([block.iloc[k]['x / m']],[block.iloc[k]['y / m']],marker='o',color='black')
            ax.text(block.iloc[k]['x / m'],block.iloc[k]['y / m']+0.2, str(int(block.iloc[k]['Pkt'])))
        block = block.append(block.iloc[0])#to close the loop
        ax.plot(block['x / m'], block['y / m'],label='Element %i' %(i+1))#,color='black')
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$y$ / m')
    ax.legend()
    fig.tight_layout()

In [ ]:
def block_area(block):
    if len(block) == 3:
        A = np.abs(block.iloc[0]['x / m'] * (block.iloc[1]['y / m'] - block.iloc[2]['y / m']) + 
                   block.iloc[1]['x / m']* (block.iloc[2]['y / m'] - block.iloc[0]['y / m']) + 
                   block.iloc[2]['x / m'] * (block.iloc[0]['y / m'] - block.iloc[1]['y / m']))/2
    elif len(block) == 4:
        A = np.abs((block.iloc[2]['x / m'] - block.iloc[0]['x / m']) * (block.iloc[3]['y / m'] - block.iloc[1]['y / m']) + 
                   (block.iloc[3]['x / m'] - block.iloc[1]['x / m']) * (block.iloc[0]['y / m'] - block.iloc[2]['y / m'])) / 2
    else:
        raise SystemExit('Element neither triangle nor quadrilateral')
    return A

In [6]:
def length(geo,ids):
    a = geo.loc[geo['Pkt']==ids[0]]
    b = geo.loc[geo['Pkt']==ids[1]]
    L = np.sqrt((a.iloc[0]['x / m'] - b.iloc[0]['x / m'])**2 + (a.iloc[0]['y / m'] - b.iloc[0]['y / m'])**2)
    return L

In [7]:
def gravity_vector(data,nr):
    return np.array([0,-data.iloc[nr-1]['G / kN/m']])

In [ ]:
def cohesion(c,geo,from_id,to_id):
    L = length(geo,[from_id,to_id])
    from_node = geo.loc[geo['Pkt']==from_id]
    to_node = geo.loc[geo['Pkt']==to_id]
    vec = np.array([to_node.iloc[0]['x / m'] - from_node.iloc[0]['x / m'], to_node.iloc[0]['y / m'] - from_node.iloc[0]['y / m']])
    vec /= np.linalg.norm(vec)
    vec *= L * c
    return vec

In [ ]:
def line_of_Q(geo,R,from_id,to_id):
    from_node = geo.loc[geo['Pkt']==from_id]
    to_node = geo.loc[geo['Pkt']==to_id]
    vec = R.dot(np.array([to_node.iloc[0]['x / m'] - from_node.iloc[0]['x / m'], to_node.iloc[0]['y / m'] - from_node.iloc[0]['y / m']]))
    vec /= np.linalg.norm(vec)
    return vec